## 路径还原-精确匹配

注：存在 86 个重复三元组，提交不知道会不会遇到这些情形
- 比如假设 triple1 即为 en 也为 zh，那么
- en -> zh -> en 和 zh -> zh -> en 都将作为答案

In [34]:
include("../src/CCKS-mKGQA.jl")
include("../src/loaddata/extractdata.jl") # 原始数据
include("../src/loaddata/translatedata.jl") # 翻译数据
include("../src/tools/ettalign.jl") # 对齐工具
include("../src/tools/distance.jl") # 距离函数
include("../src/process/submitpath.jl") # 数据提交
include("../src/datatype.jl")
mkpath("submit")

"submit"

In [35]:
# illpath = "ILLs_1"
# illpath = "ILLs_2"
# illpath = "ILLs_3"
illpath = "ILLs_4"
# illpath = "ILLs_official"
# illpath = "ILLs_wiki"
include("../src/loaddata/illsdata.jl");

### 反转字典

In [36]:
# 元组逆向
println("重复元组的信息")
sort_count(length.(values(illsdata.triple_ills2raw)))

重复元组的信息


OrderedDict{Int64, Int64} with 4 entries:
  1 => 257850
  2 => 4735
  3 => 51
  4 => 3

### 精确匹配

In [37]:
submit_id = "precise-ill3model-ill4path"
sols = split.(strip.(split(strip(read(open("predict/predict_data_0713-0.txt", "r"), String)), '\n')),'\t')
noans = "<http://dbpedia.org/resource/Francis_Russell,_Marquess_of_Tavistock>#<http://dbpedia.org/property/parents>#<http://dbpedia.org/resource/John_Russell,_4th_Duke_of_Bedford>#<http://zh.dbpedia.org/resource/第四代贝德福德公爵约翰·罗素>#<http://zh.dbpedia.org/property/successor>#<http://zh.dbpedia.org/resource/丹尼尔·芬奇，第八代温奇尔西伯爵>";

# 返回结果
res = Vector{String}(undef, 1500)
fails, notgood = Int[], Int[]
## 精确匹配
for (i, sol) in enumerate(sols)
    que, rels = ill_valid_ques[i], sol[3:end]
    ner, rawner = ill_valid_ners[i], valid_ques_ner[i][2]
    # 精确路径
    submits = precise_submit(ner, rels, illsdata)
    isempty(submits) && (push!(fails, i);continue)
    bettersubmits = filter(contains(rawner), submits)
    isempty(bettersubmits) ? push!(notgood, i) : (submits = bettersubmits)
    res[i] = first(submits)
end
println(length(fails), '\t', length(notgood))

## 剩下部分放弃
res[fails] .= noans
open("submit/submit_$submit_id.txt", "w") do io
    println(io, "id\tans_path")
    for (i, sol) in enumerate(res)
        println(io, i, '\t', sol)
    end
end

242	10


In [38]:
(0.5993 * 1500) / (1500 - 242)

0.7145866454689984

In [6]:
# ill3模型 + ill1路径
0.4673 / (1500 - 567) * 1500

0.7512861736334404

In [28]:
# ill3模型 + ill3路径
0.5247 / (1500 - 463) * 1500, ceil(0.5673 * 1500 - 0.5247 * 1500) / 463

(0.7589681774349084, 0.13822894168466524)

In [19]:
# ill3模型 + ill4路径
0.5453 / (1500 - 425) * 1500

0.7608837209302325

提交结果分析

 | ILLs | 准确率 | 完全无解 | 使用路径推理 | 缺二跳(验证集/图谱) | 缺三跳(验证集/图谱) | 
 | ---- | ----- | ----- | ----- | ------ | ---- |
 | 官方 | 0.3513 |573 | 323 | 156/52268 | 352/67437 | 
 | ILLs2 | | 551 | 221 | 98/47455 | 263/61439 | 
 | ILLs1 | 0.374 | 543 | 210 | 80/44396 | 209/57285 |
 | ILLs3 | 0.3833 | 535 | 183 | 61/37283 | 137/48256 |
 | wiki | 0.422 | 478 | 134 | 6/38070 | 103/49921 |
 | ill3 模型 + wiki 路径| 0.692 | 175 | 12 | - | - |
 | ill3 模型 + ill3 路径| 0.5673 | 301 | 162 | - | - |
 | ill3 模型 + ill4 路径| 0.5753 | 260 | 165 | - | - |
 | wiki 模型 + ill3 路径| 0.612 | - | - |
 | wiki 模型 + ill4 路径| 0.6193 | 190 | 158 | - | - |
 | wiki 模型 + wiki 路径| 0.734 | - | - | - | - |
 | ill4 模型 + ill4 路径| 0.5993 | - | - | - | - |
